In [ ]:
!pip install --upgrade gensim
from gensim.models.fasttext import FastText
import numpy as np
%tensorflow_version 1.x
import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.lancaster import LancasterStemmer
import nltk
import re
from sklearn.preprocessing import OneHotEncoder
import matplotlib.pyplot as plt
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.models import Sequential, load_model
from keras.layers import Dense, LSTM, Bidirectional, Embedding, Dropout
from keras.callbacks import ModelCheckpoint

     |████████████████████████████████| 24.2MB 1.3MB/s 
  Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0
TensorFlow 1.x selected.


Using TensorFlow backend.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
def load_dataset(filename):
  df = pd.read_csv(filename, encoding = "utf-8",
       names = ["Sentence", "Intent"])
  intent = df["Intent"]
  unique_intent = list(set(intent))
  sentences = list(df["Sentence"])
  
  return (intent, unique_intent, sentences)

In [ ]:
intent, unique_intent, sentences = load_dataset("/content/bank.csv")

In [ ]:
nltk.download("punkt")
def cleaning(sentences):
  words = [] 
  for s in sentences:
    w = word_tokenize(s)
    words.append([i for i in w])     
  return words  

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
cleaned_words = cleaning(sentences)
print(len(cleaned_words))
print(cleaned_words[:2])  
print(len(sentences))

1567
[['நான்', '2in1', 'கணக்கில்', 'சேமிப்பது', 'எப்படி', '?'], ['2in1', 'கணக்கில்', 'நான்', 'சேமிப்பது', 'எப்படி', '?']]
1567


In [ ]:
def create_tokenizer(words, filters = '!"#$%&()*+,-./:;<=>?@[\]^_`{|}~'):
  token = Tokenizer(filters = filters)
  token.fit_on_texts(words)
  return token

In [ ]:
def max_length(words):
  return(len(max(words, key = len)))

In [ ]:
word_tokenizer = create_tokenizer(cleaned_words)
vocab_size = len(word_tokenizer.word_index) + 1
max_length = max_length(cleaned_words)

print("Vocab Size = %d and Maximum length = %d" % (vocab_size, max_length))

Vocab Size = 1141 and Maximum length = 22


In [ ]:
print(cleaned_words[0])

['நான்', '2in1', 'கணக்கில்', 'சேமிப்பது', 'எப்படி', '?']


In [ ]:
ft = FastText.load("/content/drive/My Drive/ta.300/ta-d300-m2-fasttext.model")

KeyboardInterrupt: ignored

In [ ]:
print(ft.wv['2in1'])
word_tokenizer.word_index.items()

[ 3.67390402e-02 -2.66502202e-02  9.14698839e-03 -2.05807686e-02
  3.43846045e-02  3.28944996e-02 -7.55569106e-03 -1.56693091e-03
  4.21602428e-02  2.98924353e-02 -1.57219376e-02  7.92088080e-03
  1.84102915e-02  1.09176096e-02 -6.45835251e-02  1.36230411e-02
 -3.49806994e-02 -5.28973937e-02 -5.65567836e-02  3.16495858e-02
 -1.31842522e-02 -3.95152299e-03 -4.03092094e-02 -3.84472162e-02
 -7.99866952e-03 -5.05228573e-03 -3.81989405e-03 -2.98877768e-02
  2.02572104e-02 -3.52848656e-02 -9.29070264e-03 -2.83952951e-02
 -4.47124150e-03 -2.02774126e-02 -2.74483003e-02  7.05679972e-03
 -4.48973067e-02  8.17770418e-03 -5.06763253e-03 -3.17856413e-03
 -3.15665640e-02  7.49442279e-02 -1.53658148e-02  3.07027102e-02
  9.67596658e-03 -3.47672030e-02  5.47502562e-02 -8.38414580e-03
  2.74741109e-02  4.29879799e-02 -1.32845091e-02 -1.06265135e-02
 -2.83373743e-02  3.23473327e-02 -1.65915135e-02  1.32036479e-02
  2.46063508e-02  1.05681492e-03 -1.41274082e-02 -2.71935165e-02
 -4.25778050e-03  1.98123

dict_items([('?', 1), ('என்ன', 2), ('நான்', 3), ('கடன்', 4), ('எனது', 5), ('boc', 6), ('டிக்கிரி', 7), ('கணக்கை', 8), ('”', 9), ('எவ்வாறு', 10), ('எப்படி', 11), ('அட்டை', 12), ('அன்பளிப்புச்', 13), ('வெளிநாட்டு', 14), ('செலான்', 15), ('முடியும்', 16), ('அட்டையை', 17), ('நாணய', 18), ('“', 19), ('சீட்டு', 20), ('வேண்டும்', 21), ('எவை', 22), ('ஆவணங்கள்', 23), ('பெற', 24), ('.', 25), ('முடியுமா', 26), ('செய்ய', 27), ('தேவையான', 28), ('பெற்றுக்', 29), ('ஆரம்பிக்க', 30), ('fcaispe', 31), ('திறைசேரி', 32), ('சேமிப்பு', 33), ('ஆயுள்', 34), ('எந்த', 35), ('கொள்ள', 36), ('பற்றி', 37), ('எவ்வளவு', 38), ('தொழில்', 39), ('இலங்கை', 40), ('கீழ்', 41), ('நிலையான', 42), ('வழங்குனர்', 43), ('எனக்கு', 44), ('2in1', 45), ('வேண்டிய', 46), ('ஒன்றை', 47), ('புதிய', 48), ('தேவைப்படும்', 49), ('மாதாந்த', 50), ('கணக்கொன்றை', 51), ('கணக்கிற்கான', 52), ('கடனை', 53), ('குறைந்தபட்ச', 54), ('மற்றும்', 55), ('வகையின்', 56), ('வயது', 57), ('தொகை', 58), ('விற்பனை', 59), ('பெறுவது', 60), ('வேண்டுமா', 61), ('கணக்கில்', 6

In [ ]:
word_index =word_tokenizer.word_index

In [ ]:
#embedding matrix

print('preparing embedding matrix...')
words_not_found = []
nb_words = 1141

embedding_matrix = np.zeros((nb_words, embedding_size ))
for word, i in word_index.items():
    if i >= nb_words:
        continue
    embedding_vector = ft.wv[word]
    if (embedding_vector is not None) and len(embedding_vector) > 0:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector
    else:
        words_not_found.append(word)
print('number of null word embeddings: %d' % np.sum(np.sum(embedding_matrix, axis=1) == 0))

preparing embedding matrix...
number of null word embeddings: 1


In [ ]:
print("sample words not found: ", words_not_found)

sample words not found:  []


In [ ]:
def encoding_doc(token, words):
  return(token.texts_to_sequences(words))

In [ ]:
#tokenizer with filter changed
output_tokenizer = create_tokenizer(unique_intent, filters = '!"#$%&()*+,-/:;<=>?@[\]^`{|}~')
encoded_output = encoding_doc(output_tokenizer, intent)
encoded_output = np.array(encoded_output).reshape(len(encoded_output), 1)
encoded_output.shape


(1567, 1)

In [ ]:
def one_hot(encode):
  o = OneHotEncoder(sparse = False)
  return(o.fit_transform(encode))

In [ ]:
output_one_hot = one_hot(encoded_output)

In [ ]:
encoded_doc = encoding_doc(word_tokenizer, cleaned_words)

In [ ]:
def padding_doc(encoded_doc, max_length):
  return(pad_sequences(encoded_doc, maxlen = max_length, padding = "post"))

In [ ]:
padded_doc = padding_doc(encoded_doc, max_length)

In [ ]:
from sklearn.model_selection import train_test_split
train_X, val_X, train_Y, val_Y = train_test_split(padded_doc, output_one_hot, shuffle = True, test_size = 0.2)
print("Shape of train_X = %s and train_Y = %s" % (train_X.shape, train_Y.shape))
print("Shape of val_X = %s and val_Y = %s" % (val_X.shape, val_Y.shape))

Shape of train_X = (1253, 22) and train_Y = (1253, 56)
Shape of val_X = (314, 22) and val_Y = (314, 56)


In [ ]:
from keras.layers import Embedding, Conv1D, MaxPooling1D, GlobalMaxPooling1D 
from keras import regularizers
from keras import optimizers

batch_size = 256 
num_epochs = 8 

#model parameters
num_filters = 64 
#embed_dim = 300 
weight_decay = 1e-4

num_classes = 56
print("training CNN ...")
model = Sequential()
model.add(Embedding(nb_words, embedding_size, weights=[embedding_matrix], input_length=max_length, trainable=False))
model.add(Conv1D(num_filters, 7, activation='linear', padding='same'))
model.add(MaxPooling1D(2))
model.add(Conv1D(num_filters, 7, activation='linear', padding='same'))
model.add(GlobalMaxPooling1D())
model.add(Dropout(0.5))
model.add(Dense(32, activation='linear', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Dense(num_classes, activation='sigmoid'))  #multi-label (k-hot encoding)

adam = optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])
model.summary()


training CNN ...
Instructions for updating:
If using Keras pass *_constraint arguments to layers.

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 22, 300)           342300    
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 22, 64)            134464    
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 11, 64)            0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 11, 64)            28736     
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 64)                0         
___________________________________________________________

In [ ]:
from keras.callbacks import EarlyStopping
#define callbacks
early_stopping = EarlyStopping(monitor='val_loss', min_delta=0.01, patience=4, verbose=1)
callbacks_list = [early_stopping]

In [ ]:
hist = model.fit(train_X, train_Y, batch_size=batch_size, epochs=num_epochs, callbacks=callbacks_list, validation_split=0.1, shuffle=True, verbose=2)


Train on 1127 samples, validate on 126 samples
Epoch 1/8
 - 1s - loss: 0.6593 - accuracy: 0.6246 - val_loss: 0.5729 - val_accuracy: 0.7275
Epoch 2/8
 - 0s - loss: 0.5636 - accuracy: 0.7250 - val_loss: 0.4364 - val_accuracy: 0.7942
Epoch 3/8
 - 0s - loss: 0.4749 - accuracy: 0.7920 - val_loss: 0.3229 - val_accuracy: 0.9005
Epoch 4/8
 - 0s - loss: 0.3877 - accuracy: 0.8493 - val_loss: 0.2392 - val_accuracy: 0.9510
Epoch 5/8
 - 0s - loss: 0.3077 - accuracy: 0.8939 - val_loss: 0.1775 - val_accuracy: 0.9663
Epoch 6/8
 - 0s - loss: 0.2368 - accuracy: 0.9305 - val_loss: 0.1384 - val_accuracy: 0.9809
Epoch 7/8
 - 0s - loss: 0.1830 - accuracy: 0.9586 - val_loss: 0.1204 - val_accuracy: 0.9821
Epoch 8/8
 - 0s - loss: 0.1539 - accuracy: 0.9717 - val_loss: 0.1110 - val_accuracy: 0.9821


In [ ]:
print(model.test_on_batch(val_X,val_Y))
model.metrics_names

[0.11559468, 0.98214304]


['loss', 'accuracy']

In [ ]:
from imblearn.over_sampling import SMOTE

/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


In [ ]:
train_X.shape,train_Y.shape


((1253, 22), (1253, 56))

In [ ]:
smote = SMOTE('minority')
X_sm, y_sm = smote.fit_sample(train_X,train_Y)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
print(X_sm.shape,y_sm.shape)

(1313, 22) (1313, 56)


In [ ]:
hist = model.fit(X_sm, y_sm, batch_size=batch_size, epochs=num_epochs, callbacks=callbacks_list, validation_split=0.1, shuffle=True, verbose=2)

Train on 1181 samples, validate on 132 samples
Epoch 1/8
 - 0s - loss: 0.1349 - accuracy: 0.9780 - val_loss: 0.1159 - val_accuracy: 0.9821
Epoch 2/8
 - 0s - loss: 0.1213 - accuracy: 0.9803 - val_loss: 0.1075 - val_accuracy: 0.9821
Epoch 3/8
 - 0s - loss: 0.1143 - accuracy: 0.9814 - val_loss: 0.1005 - val_accuracy: 0.9821
Epoch 4/8
 - 0s - loss: 0.1092 - accuracy: 0.9815 - val_loss: 0.0967 - val_accuracy: 0.9821
Epoch 5/8
 - 0s - loss: 0.1067 - accuracy: 0.9818 - val_loss: 0.0944 - val_accuracy: 0.9821
Epoch 6/8
 - 0s - loss: 0.1032 - accuracy: 0.9820 - val_loss: 0.0931 - val_accuracy: 0.9821
Epoch 7/8
 - 0s - loss: 0.1015 - accuracy: 0.9821 - val_loss: 0.0922 - val_accuracy: 0.9821
Epoch 00007: early stopping


In [ ]:
print(model.test_on_batch(val_X,val_Y))
model.metrics_names

[0.09192205, 0.982143]


['loss', 'accuracy']

In [ ]:
from sklearn.utils import class_weight
y_train_labels = np.argmax(train_Y, axis =1)
class_weight = class_weight.compute_class_weight('balanced',np.unique(y_train_labels),y_train_labels)

In [ ]:
hist = model.fit(X_sm, y_sm, batch_size=batch_size, epochs=num_epochs, class_weight=class_weight, callbacks=callbacks_list, validation_split=0.1, shuffle=True, verbose=2)

Train on 1181 samples, validate on 132 samples
Epoch 1/8
 - 0s - loss: 0.1000 - accuracy: 0.9821 - val_loss: 0.0915 - val_accuracy: 0.9821
Epoch 2/8
 - 0s - loss: 0.0976 - accuracy: 0.9821 - val_loss: 0.0915 - val_accuracy: 0.9821
Epoch 3/8
 - 0s - loss: 0.0968 - accuracy: 0.9821 - val_loss: 0.0918 - val_accuracy: 0.9821
Epoch 4/8
 - 0s - loss: 0.0952 - accuracy: 0.9821 - val_loss: 0.0922 - val_accuracy: 0.9821
Epoch 5/8
 - 0s - loss: 0.0944 - accuracy: 0.9821 - val_loss: 0.0922 - val_accuracy: 0.9821
Epoch 00005: early stopping


In [ ]:
print(model.test_on_batch(val_X,val_Y))
model.metrics_names

[0.0883926, 0.982143]


['loss', 'accuracy']

In [ ]:
hist = model.fit(train_X, train_Y, batch_size=batch_size, epochs=num_epochs, class_weight=class_weight, callbacks=callbacks_list, validation_split=0.1, shuffle=True, verbose=2)

Train on 1127 samples, validate on 126 samples
Epoch 1/8
 - 0s - loss: 0.0928 - accuracy: 0.9822 - val_loss: 0.0841 - val_accuracy: 0.9821
Epoch 2/8
 - 0s - loss: 0.0924 - accuracy: 0.9822 - val_loss: 0.0839 - val_accuracy: 0.9821
Epoch 3/8
 - 0s - loss: 0.0925 - accuracy: 0.9821 - val_loss: 0.0831 - val_accuracy: 0.9821
Epoch 4/8
 - 0s - loss: 0.0909 - accuracy: 0.9822 - val_loss: 0.0828 - val_accuracy: 0.9821
Epoch 5/8
 - 0s - loss: 0.0909 - accuracy: 0.9822 - val_loss: 0.0820 - val_accuracy: 0.9821
Epoch 00005: early stopping


In [ ]:
print(model.test_on_batch(val_X,val_Y))
model.metrics_names

[0.085677214, 0.98214304]


['loss', 'accuracy']

In [ ]:
model.evaluate(val_X,val_Y)

314/314 [==============================] - 0s 208us/step


[0.08567721553289206, 0.9821428656578064]